In [1]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [32]:
EMBEDDINGS_PATH = '../../embeddings/visual_embeddings/'
EMBEDDINGS_FILES = os.listdir(EMBEDDINGS_PATH)
embeddings_map = {}
for file in EMBEDDINGS_FILES:
    with open(EMBEDDINGS_PATH + file, 'rb') as f:
        embeddings_map[file.split('.')[0]] = pickle.load(f).squeeze()
with open('../../embeddings/visual_features.pkl', 'wb') as f:
    pickle.dump(embeddings_map, f)

In [33]:
TRUTHFUL_PATH = '../../data/Clips/Truthful/'
DECEPTIVE_PATH = '../../data/Clips/Deceptive/'
TRUTHFUL_FILES = os.listdir(TRUTHFUL_PATH)
DECEPTIVE_FILES = os.listdir(DECEPTIVE_PATH)

In [34]:
class VisualClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(VisualClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 64)
        self.linear2 = torch.nn.Linear(64, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [35]:
class VisualClassifierPCA(torch.nn.Module):
    def __init__(self, embedding_size):
        super(VisualClassifierPCA, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        # Xavier initialize the linear layer
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [36]:
class VisualDataset(Dataset):
    def __init__(self, data, embeddings_file):
        with open(embeddings_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.data = [embeddings_map[file.split('.')[0]] for file in data]
        self.labels = [0 if file.split('_')[1] == 'truth' else 1 for file in data]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [52]:
class VisualDatasetPCA(Dataset):
    def __init__(self, data, embeddings_file, pca, sc, test=False):
        with open(embeddings_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.data = np.array([embeddings_map[file.split('.')[0]].numpy() for file in data])
        if not test:
            self.data = sc.fit_transform(self.data)
            self.data = pca.fit_transform(self.data)
        else:
            self.data = sc.transform(self.data)
            self.data = pca.transform(self.data)
        self.data = torch.from_numpy(self.data).float()
        self.labels = [0 if file.split('_')[1] == 'truth' else 1 for file in data]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [41]:
def eval(model, val_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [42]:
def kfold(embeddings_map_file, train_batch_size, do_pca, embedding_size):
    truthful_data = list(TRUTHFUL_FILES)
    deceptive_data = list(DECEPTIVE_FILES)
    for i in range(10):
        val_data = truthful_data[i*6:(i+1)*6] + deceptive_data[i*6:(i+1)*6]
        train_data = truthful_data[:i*6] + truthful_data[(i+1)*6:] + deceptive_data[:i*6] + deceptive_data[(i+1)*6:]
        if not do_pca:
            train_dataset = VisualDataset(train_data, embeddings_map_file)
            val_dataset = VisualDataset(val_data, embeddings_map_file)
        else:
            sc = StandardScaler()
            pca = PCA(n_components=embedding_size)
            train_dataset = VisualDatasetPCA(train_data, embeddings_map_file, pca, sc)
            val_dataset = VisualDatasetPCA(val_data, embeddings_map_file, pca, sc, test=True)
        train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=12)
        yield train_loader, val_loader

In [43]:
def run_one_fold(train_loader, val_loader, num_epochs, embedding_size, do_pca):
    if not do_pca:
        model = VisualClassifier(embedding_size)
    else:
        model = VisualClassifierPCA(embedding_size)
    model.to(device)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs)
    return eval(model, val_loader)

In [44]:
def get_model_accuracy(embeddings_map_file, train_batch_size, num_epochs, embedding_size = 512, do_pca = False):
    accuracies = []
    for train_loader, val_loader in kfold(embeddings_map_file, train_batch_size, do_pca, embedding_size):
        accuracies.append(run_one_fold(train_loader, val_loader, num_epochs, embedding_size, do_pca))
    return sum(accuracies) / len(accuracies)

In [45]:
# Model accuracy for resnet-3d-18
get_model_accuracy('../../embeddings/visual_features.pkl', 4, 40)

0.5

In [55]:
# Model accuracy for resnet-3d-18
get_model_accuracy('../../embeddings/visual_features.pkl', 4, 20)

0.5

In [53]:
# Model accuracy for resnet-3d-18 with pca
get_model_accuracy('../../embeddings/visual_features.pkl', 4, 40, 64, True)

0.5916666666666666

In [54]:
# Model accuracy for resnet-3d-18 with pca
get_model_accuracy('../../embeddings/visual_features.pkl', 4, 20, 64, True)

0.575

In [56]:
# Model accuracy for resnet-3d-18 with pca
get_model_accuracy('../../embeddings/visual_features.pkl', 2, 40, 64, True)

0.5833333333333334